In [35]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

from pathlib import Path

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

model = timm.create_model('mobilenetv3_large_100.ra_in1k', pretrained=True, num_classes = 2)
model.requires_grad_(False)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')
model.to(device)

image_size = 224

num_epochs = 10

/home/mosminin/dev/pytorch-image-models/.venv/lib/python3.9/site-packages/safetensors/torch.py:99: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
/home/mosminin/dev/pytorch-image-models/.venv/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/home/mosminin/dev/pytorch-image-models/.venv/lib/python3.9/site-packages/torch/storage.py:899: UserWarning: TypedStorage is d

### Dataloader

In [36]:
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.RandomErasing(),
    transforms.RandAugment(),
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

valid_transforms = transforms.Compose([
    transforms.RandomRotation(5),
    transforms.RandomErasing(),
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [37]:
# classes = ('something', 'person')

data_path = str(Path('/home/mosminin/dev/yolo4scaled/darknet/parser/datasets/COCO2017/person_class'))

train_dataset = ImageFolder(root=data_path+'/train', transform=train_transforms)
valid_dataset = ImageFolder(root=data_path+'/valid', transform=valid_transforms)

# # Create datasets for training & validation, download if necessary
# train_dataset = torchvision.datasets.FashionMNIST('./data', train=True, transform=train_transforms, download=True)
# valid_dataset = torchvision.datasets.FashionMNIST('./data', train=False, transform=valid_transforms, download=True)

In [45]:
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=1)
validloader = DataLoader(valid_dataset, batch_size=32, shuffle=True, num_workers=1)

In [46]:
trainloader.dataset

Dataset ImageFolder
    Number of datapoints: 114758
    Root location: /home/mosminin/dev/yolo4scaled/darknet/parser/datasets/COCO2017/person_class/train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-5.0, 5.0], interpolation=nearest, expand=False, fill=0)
               RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)
               RandAugment(num_ops=2, magnitude=9, num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

### Train @ Val

In [49]:
for epoch in range(num_epochs):
    train_loss = 0.0
    valid_loss = 0.0
    train_correct = 0
    valid_correct = 0

    # Тренировочный этап
    model.train()
    for data, target in trainloader:
        try:
            data = data.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()*data.size(0)
            _, preds = torch.max(output, 1)
            train_correct += torch.sum(preds == target.data)
        except AttributeError:
            continue
    train_loss = train_loss/len(trainloader.dataset)
    train_acc = train_correct.double()/len(trainloader.dataset)

    # Валидационный этап
    model.eval()
    for data, target in validloader:
        try:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            loss = criterion(output, target)
            valid_loss += loss.item()*data.size(0)
            _, preds = torch.max(output, 1)
            valid_correct += torch.sum(preds == target.data)
        except AttributeError:
            continue
    valid_loss = valid_loss/len(validloader.dataset)
    valid_acc = valid_correct.double()/len(validloader.dataset)

    # Выводим результаты после каждой эпохи
    print('Epoch: {} \tTraining Loss: {:.6f} \tTraining Accuracy: {:.6f} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.6f}'.format(
        epoch+1, train_loss, train_acc, valid_loss, valid_acc))

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/mosminin/dev/pytorch-image-models/.venv/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/mosminin/dev/pytorch-image-models/.venv/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/mosminin/dev/pytorch-image-models/.venv/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/mosminin/dev/pytorch-image-models/.venv/lib/python3.9/site-packages/torchvision/datasets/folder.py", line 231, in __getitem__
    sample = self.transform(sample)
  File "/home/mosminin/dev/pytorch-image-models/.venv/lib/python3.9/site-packages/torchvision/transforms/transforms.py", line 95,

### test